### HEROKUのポスグレの接続テスト

In [232]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# カーソルを取得する
cursor = conn.cursor()



#自動コミットモードにする

# conn.autocommit = True
# conn.autocommit

In [234]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度 , "住所(ホールデータ)"
               FROM tenpo_master
               WHERE 都道府県 = '神奈川県' 
               AND 経度 is NULL 
               OR 都道府県 = '東京都'
               AND 経度 is NULL
                OR 都道府県 = '埼玉県'
               AND 経度 is NULL
                OR 都道府県 = '千葉県'
               AND 経度 is NULL
               ORDER BY 都道府県''')
cols = [col.name for col in cursor.description]
df = pd.DataFrame(cursor.fetchall(),columns=cols)
df

,店舗名／法人名,都道府県,緯度,経度,住所(ホールデータ)


In [233]:
cursor.execute('''DELETE FROM tenpo_master
               WHERE 都道府県 = '神奈川県' 
               AND 経度 is NULL 
               OR 都道府県 = '東京都'
               AND 経度 is NULL
                OR 都道府県 = '埼玉県'
               AND 経度 is NULL
                OR 都道府県 = '千葉県'
               AND 経度 is NULL''')
conn.commit()

In [235]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度
               FROM tenpo_master
               WHERE 経度 is NULL ''')
cols = [col.name for col in cursor.description]
df = pd.DataFrame(cursor.fetchall(),columns=cols)
df

,店舗名／法人名,都道府県,緯度,経度
0,ひまわり北上店,岩手県,None,None
1,グランキコーナ泉佐野店,大阪府,None,None
2,パチンコジャンボ,,None,None
3,APAN CLUB トガリ店,愛知県,None,None
4,メガガイア上越,新潟県,None,None
...,...,...,...,...
208,トキワホール,,None,None
209,パーラーデルダス,,None,None
210,Super D’station前橋大利根店,群馬県,None,None
211,yout小牧店,愛知県,None,None


In [190]:
dmm_tenpo_name

'ＳＬＯＴ\u3000Ｇ８'

In [222]:
dmm_tenpo_name.replace("'", "''")

"Super D''station錦糸町店"

In [225]:
for i,row in serched_df.iterrows():
    print(row['longtitude'])
    longtitude = row['longtitude']
    latitude = row['latitude']
    dmm_tenpo_name = row['dmm_tenpo_name'].replace("'", "''")
    try:
        cursor.execute(f"""UPDATE tenpo_master
                    SET 経度 = {longtitude}, 緯度 = {latitude}
                WHERE  店舗名／法人名 = '{dmm_tenpo_name}' """)
        conn.commit()
    except Exception as e: 
        print("error", dmm_tenpo_name)
        break



139.880066
140.116348
140.021362
139.937988
139.433853
139.802078
139.318695
139.619141
139.719803
139.705093
139.096313
139.65303
139.789978
139.408783
139.688934
139.491028
139.472565
139.655518
139.857559
139.35994
139.736267
139.790375
139.834
139.717804
139.270264
139.816986
139.926605
139.444992
139.790894
139.382294
139.669434
139.446503
139.804565
139.791916
136.907288
136.907288
139.79509
140.586578
139.431732
139.698776
139.701553
139.318069
139.466766
139.436432
139.567551
139.622421
139.619217
139.671021
139.390137
139.204315
139.395065
139.579208
139.579208
139.680878
139.633621
139.184937
139.630905


In [194]:
cursor.execute(f'''UPDATE tenpo_master
                SET 経度 = {longtitude}, 緯度 = {latitude}
               WHERE  店舗名／法人名 = 'ＳＬＯＴ Ｇ８' ''')

In [171]:
cursor.execute(f'''SELECT 店舗名／法人名, 都道府県,  緯度, 経度 , "住所(ホールデータ)"
               FROM tenpo_master
               WHERE  店舗名／法人名 = 'エムズファクトリー(千葉)' ''')
cols = [col.name for col in cursor.description]
df = pd.DataFrame(cursor.fetchall(),columns=cols)
df

,店舗名／法人名,都道府県,緯度,経度,住所(ホールデータ)
0,エムズファクトリー(千葉),千葉県,None,None,


In [ ]:
serched_df = pd.read_csv('csv/scrapinged_kanto_tenpo_name_table.csv')
serched_df

In [ ]:

cursor.execute('''SELECT *
               FROM maptable
               WHERE prefecture = '神奈川県' AND latitude is NULL ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)





In [ ]:
cursor = conn.cursor()
cursor.execute('''SELECT COUNT(DISTINCT 店舗名)
               FROM parlar_data
               WHERE 都道府県 = '神奈川県' or '埼玉県' ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)

In [ ]:
cursor.execute('''SELECT * 
               FROM schedule 
               ORDER BY 取得時間 DESC
               LIMIT 10''')
cols = [col.name for col in cursor.description]
schedule_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(schedule_df)